# Index calculus algorithm

We will perform the Index calculus algorithm to compute $\log_{a} b_i$ ($i = 1, 2$) in $\mathbf{Z}_{p}^*$, where $a = 5$, $b_1 = 4389733$, $b_2 = 1234567$, and $p = 9330887$ is a prime number.

In [1]:
from algorithms.euclidean import gcd
from algorithms.factorization import factorizeByBase

p = 9330887
a = 5
b1 = 4389733
b2 = 1234567

We will use a base consisting of $-1$ and all primes less than $50$.

In [2]:
base = [-1] + prime_range(50)

Let us now try to find the logarithms table. We construct a matrix by trying random powers of $a$ and factoring them with the numbers in our base. We stop when the matrix has full rank.

In [3]:
set_random_seed(0)

v = []
s = set()
r, l = 0, len(base)
M = Matrix(nrows=0, ncols=l)
while r < l:
    i = randint(1, p-1)
    if i in s:
        continue
    s.add(i)
    x = pow(a, i, p)
    f = factorizeByBase(Integer(x), base, p)
    if not f:
        continue
    MM = Matrix(list(M) + [f])
    rr = MM.rank()
    if rr > r:
        M = MM
        r = rr
        v.append(i)
print(M)
print(v)

[1 0 1 0 1 0 1 1 0 0 0 0 1 0 1 0]
[1 1 4 0 2 0 0 0 1 1 0 0 0 0 0 0]
[1 1 5 2 1 0 0 1 0 0 0 0 0 0 0 0]
[1 6 0 0 0 1 1 0 0 0 0 0 0 1 0 0]
[0 3 1 0 0 1 0 0 0 0 1 1 0 0 0 0]
[0 2 0 2 0 0 1 0 0 0 1 0 0 0 0 1]
[1 9 3 2 1 0 0 0 0 0 0 0 0 0 0 0]
[0 1 1 2 0 0 0 0 0 0 1 1 0 1 0 0]
[0 3 4 1 0 0 1 0 1 0 0 0 0 0 0 0]
[0 1 0 0 3 0 0 0 1 0 0 0 0 0 0 1]
[0 3 3 0 0 0 0 0 2 0 0 0 0 0 1 0]
[1 2 0 0 0 2 0 0 0 0 0 1 0 1 0 0]
[1 6 3 0 1 0 0 2 0 0 0 0 0 0 0 0]
[1 0 0 1 2 0 0 0 0 1 2 0 0 0 0 0]
[1 6 0 0 0 1 1 0 0 0 1 1 0 0 0 0]
[0 3 1 1 1 0 1 0 1 0 0 0 1 0 0 0]
[4746917, 4747026, 6889022, 3908343, 1066760, 6494367, 1934471, 1567977, 7864629, 1638582, 2526531, 6829987, 6466117, 3166568, 4256144, 5639062]


We now solve the system of equations we have obtained. The solution represents our table of logarithms and can be used to find logarithms of multiple numbers.

In [4]:
t = M^-1 * Matrix(zip(v)) % (p-1)
t

[4665443]
[6670912]
[2030334]
[      1]
[5786904]
[1078197]
[8534197]
[7606749]
[8519903]
[2519168]
[6200403]
[9068634]
[7409417]
[5590350]
[6037417]
[6410599]

Let us verify that the table is correct.

In [5]:
[pow(a, i, p) for i, in t]

[9330886, 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]

We can now find a power of $b_i$ that can be factorized with our base. We will use the following function.

In [6]:
def findPower(b, p, t, base):
    while True:
        i = randint(1, p-1)
        x = pow(b, i, p)
        f = factorizeByBase(Integer(x), base, p)
        if not f:
            continue
        return (i, f)

Let us find such a power of $b_1$ and its factorization.

In [7]:
i1, f1 = findPower(b1, p, t, base)
i1, f1

(7675068, [0, 1, 3, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0])

We can now use the logarithm table to obtain a congruence in $\log_a b_1$. However, there may be multiple solutions, and we need to check which one is our answer. The number of solutions is given by the greatest common divisor of $p-1$ and the obtained exponent.

In [8]:
g1 = gcd(i1, p-1)
g1

2

We can thus obtain a solution modulo $p-1$ divided by this GCD.

In [9]:
m1 = ((p-1)/g1)
(x1,), = Matrix([f1])*t / i1 % m1
x1

1087862

Let us now put this into a function which will also verify the potential solutions.

In [10]:
def checkSolutions(a, b, p, t, i, f):
    g = gcd(i, p-1)
    m = (p-1)/g
    (x,), = Matrix([f])*t / i % m
    for j in range(g):
        y = pow(a, x, p)
        print("%d^%d mod %d = %d" % (a, x, p, y))
        if y == b:
            return x
        x += m

checkSolutions(a, b1, p, t, i1, f1)

5^1087862 mod 9330887 = 4941154
5^5753305 mod 9330887 = 4389733


5753305

We have thus computed $\log_a b_1$. Let us now compute $\log_a b_2$.

In [11]:
i2, f2 = findPower(b2, p, t, base)
i2, f2

(1643897, [1, 7, 2, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [12]:
checkSolutions(a, b2, p, t, i2, f2)

5^7273774 mod 9330887 = 1234567


7273774

We will now use the function `logarithmTable` to compute tables of logarithms, which will then be used to compute some more discrete logarithms with the function `indexCalculus`. We will measure the evaluation times.

In [13]:
from algorithms.discreteLogarithm import logarithmTable, indexCalculus

aa = 47
bb = 191

In [14]:
pp = 100000000003
table = %time logarithmTable(aa, pp, base, trace = True)
print(table)
%time indexCalculus(aa, bb, pp, base, table, trace = True)

found factorization 47^29227416932 = 2^4 * 3^2 * 5^1 * 11^1 * 13^1 * 17^1 * 23^1 * 37^2 (mod 100000000003)
found factorization 47^66046858944 = -1^1 * 2^2 * 3^2 * 5^2 * 7^3 * 17^1 * 19^2 * 23^1 (mod 100000000003)
found factorization 47^24486550609 = 3^2 * 7^1 * 11^1 * 17^3 * 29^1 * 37^1 (mod 100000000003)
found factorization 47^85242779496 = 2^4 * 3^1 * 5^4 * 17^4 * 37^1 (mod 100000000003)
found factorization 47^16805509848 = -1^1 * 3^3 * 5^3 * 7^1 * 17^1 * 29^1 * 41^1 * 43^1 (mod 100000000003)
found factorization 47^75591456706 = -1^1 * 2^8 * 3^4 * 7^2 * 11^2 * 19^1 * 29^1 (mod 100000000003)
found factorization 47^8278721305 = -1^1 * 2^2 * 3^3 * 11^1 * 19^1 * 23^2 * 29^2 (mod 100000000003)
found factorization 47^58912326141 = 3^2 * 5^1 * 7^2 * 13^1 * 29^3 * 37^1 (mod 100000000003)
found factorization 47^38191145701 = 2^5 * 3^1 * 11^1 * 13^1 * 17^2 * 19^2 * 31^1 (mod 100000000003)
found factorization 47^37063051674 = 2^4 * 5^1 * 13^4 * 17^2 * 19^1 (mod 100000000003)
found factorization

6935101882

In [15]:
pp = 10000000000000000051
base = [-1] + prime_range(5000)
table = %time logarithmTable(aa, pp, base, trace = True)
%time indexCalculus(aa, bb, pp, base, table, trace = True)

found factorization 47^7656292675090614622 = -1^1 * 2^7 * 3^1 * 97^1 * 313^1 * 487^1 * 491^1 * 1579^1 * 2131^1 (mod 10000000000000000051)
found factorization 47^5897058151425594450 = 2^3 * 23^1 * 37^2 * 71^1 * 79^1 * 139^1 * 1637^1 * 2557^1 (mod 10000000000000000051)
found factorization 47^9164593694298065188 = -1^1 * 2^1 * 3^1 * 5^1 * 11^1 * 13^1 * 67^1 * 191^1 * 227^1 * 439^1 * 599^1 * 2671^1 (mod 10000000000000000051)
found factorization 47^7922551988556957951 = 2^4 * 3^1 * 5^1 * 37^1 * 47^1 * 59^1 * 83^1 * 1583^1 * 1609^1 * 1847^1 (mod 10000000000000000051)
found factorization 47^1033256369410264427 = -1^1 * 2^1 * 7^1 * 23^1 * 1291^1 * 1429^1 * 1601^1 * 2777^1 * 3533^1 (mod 10000000000000000051)
found factorization 47^2184518400175163626 = 2^3 * 5^2 * 79^1 * 263^1 * 269^1 * 919^1 * 2689^1 * 2861^1 (mod 10000000000000000051)
found factorization 47^6644364455943886375 = -1^1 * 2^1 * 13^1 * 43^1 * 97^1 * 107^1 * 179^1 * 431^1 * 929^1 * 4463^1 (mod 10000000000000000051)
found factoriza

found factorization 47^4587355764029354492 = 3^2 * 5^1 * 19^2 * 29^1 * 97^1 * 241^2 * 1373^1 * 1381^1 (mod 10000000000000000051)
found factorization 47^1064503433085610551 = 3^1 * 53^1 * 409^1 * 613^1 * 757^1 * 3923^1 * 4957^1 (mod 10000000000000000051)
found factorization 47^3768282425562682518 = -1^1 * 3^1 * 79^2 * 197^1 * 379^1 * 613^1 * 1297^1 * 3853^1 (mod 10000000000000000051)
found factorization 47^7040398255111055417 = 3^2 * 7^3 * 457^1 * 631^1 * 1129^1 * 2437^1 * 3607^1 (mod 10000000000000000051)
found factorization 47^3638881606240851367 = -1^1 * 3^1 * 5^1 * 139^1 * 317^1 * 353^1 * 2351^1 * 3457^1 * 4421^1 (mod 10000000000000000051)
found factorization 47^1774809294437377104 = 2^3 * 3^1 * 103^1 * 521^2 * 541^1 * 3967^1 * 4211^1 (mod 10000000000000000051)
found factorization 47^4725811024570028178 = -1^1 * 3^1 * 137^1 * 163^1 * 239^1 * 349^1 * 941^1 * 977^1 * 1259^1 (mod 10000000000000000051)
found factorization 47^7326969226354823496 = 2^1 * 3^1 * 5^3 * 11^1 * 13^1 * 17^1 * 7

KeyboardInterrupt: 